In [26]:
!pip install pyspark

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [28]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

But de cette session:
1. Explorer la donnée
2. Préparer la donnée
3. Entrainer un ou deux modèles de ML (prévision du prix)
4. Etendre au 2è jeu de données

In [29]:
#l_listings = pd.read_csv("listings_lyon.csv")
l_listings = pd.read_csv("listings.csv")

Recherche de l'assymétrie de la distribution des prix
Si elle est positive, les points aberrants sont plutôt les locations chères

In [30]:
l_listings['price'].skew()

38.913407111565995

In [31]:
p_listings = pd.read_csv("listings_paris.csv")
p_listings['price'].skew()

31.237462467794693

In [32]:
l_listings.describe(include='all')

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
count,1.011300e+04,10113,1.011300e+04,10111,0.0,10113,10113.000000,10113.000000,10113,5697.000000,10113.000000,10113.000000,8076,8076.000000,10113.000000,10113.000000,10113.000000,5885
unique,NaN,9796,NaN,2388,NaN,9,NaN,NaN,4,NaN,NaN,NaN,1579,NaN,NaN,NaN,NaN,5114
top,NaN,Welcome To Vaise,NaN,David,NaN,3e Arrondissement,NaN,NaN,Entire home/apt,NaN,NaN,NaN,2024-03-17,NaN,NaN,NaN,NaN,"Available with a mobility lease only (""bail mo..."
freq,NaN,11,NaN,161,NaN,1804,NaN,NaN,7901,NaN,NaN,NaN,525,NaN,NaN,NaN,NaN,280
mean,3.128645e+17,NaN,1.409049e+08,NaN,NaN,NaN,45.759098,4.840359,NaN,112.081797,77.166815,35.209236,NaN,1.272360,8.188174,105.805102,9.123702,NaN
std,4.292113e+17,NaN,1.589800e+08,NaN,NaN,NaN,0.013107,0.019979,NaN,226.023310,146.292246,78.117394,NaN,1.736275,23.022586,127.159838,18.057338,NaN
min,5.676600e+04,NaN,4.382300e+04,NaN,NaN,NaN,45.721930,4.773840,NaN,12.000000,1.000000,0.000000,NaN,0.010000,1.000000,0.000000,0.000000,NaN
25%,1.988972e+07,NaN,2.393238e+07,NaN,NaN,NaN,45.750847,4.828670,NaN,59.000000,1.000000,1.000000,NaN,0.140000,1.000000,0.000000,0.000000,NaN
50%,4.071661e+07,NaN,6.356759e+07,NaN,NaN,NaN,45.759170,4.836860,NaN,80.000000,2.000000,7.000000,NaN,0.560000,1.000000,42.000000,1.000000,NaN
75%,7.661984e+17,NaN,2.194900e+08,NaN,NaN,NaN,45.769050,4.852580,NaN,120.000000,14.000000,31.000000,NaN,1.752500,2.000000,203.000000,10.000000,NaN


In [33]:
l_listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,56766,"Duplex avec vue, centre ville",269557,Isabelle,NaN,5e Arrondissement,45.76312,4.82813,Entire home/apt,125.0,60,55,2023-10-29,0.34,1,210,1,"Available with a mobility lease only (""bail mo..."
1,77104,Nice Flat in the center of Lyon,411285,Clementine,NaN,1er Arrondissement,45.76842,4.82745,Entire home/apt,100.0,2,24,2023-10-07,0.15,1,75,2,6938113058353
2,134539,"Centre of old town,home sweet home",660135,Polali,NaN,5e Arrondissement,45.75930,4.82548,Entire home/apt,80.0,2,132,2023-12-10,0.85,1,82,6,6938513221658
3,174304,Quartier Terreaux 4/6 pers. Hyper centre/Clim,832917,Giuseppina,NaN,1er Arrondissement,45.76695,4.83241,Entire home/apt,201.0,2,429,2024-03-10,3.02,2,115,42,6938112368465
4,233372,Clear &very central flat Vieux Lyon,1221417,Max,NaN,5e Arrondissement,45.75935,4.82228,Entire home/apt,52.0,2,316,2024-03-21,2.09,1,218,64,6938512680042


In [34]:
l_listings.isnull().sum()

id                                    0
name                                  0
host_id                               0
host_name                             2
neighbourhood_group               10113
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                              4416
minimum_nights                        0
number_of_reviews                     0
last_review                        2037
reviews_per_month                  2037
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                            4228
dtype: int64

In [35]:
list(set(l_listings['room_type']))

['Entire home/apt', 'Hotel room', 'Shared room', 'Private room']

In [36]:
spark = SparkSession.builder.appName('FilRouge').getOrCreate()

In [37]:
df_l_listings = spark.createDataFrame(l_listings)

In [38]:
df_l_listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: long (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: double (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: double (nullable = true)
 |-- minimum_nights: long (nullable = true)
 |-- number_of_reviews: long (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: double (nullable = true)
 |-- calculated_host_listings_count: long (nullable = true)
 |-- availability_365: long (nullable = true)
 |-- number_of_reviews_ltm: long (nullable = true)
 |-- license: string (nullable = true)



In [53]:
df_l_listing_filtered = df_l_listings.filter((df_l_listings['price']>50)&(df_l_listings['price']<1000))

In [40]:
col_drop = ['name', 'host_name', 'neighbourhood_group', 'neighbourhood',
            'calculated_host_listings_count', 'license']
df_l_listing_filtered = df_l_listing_filtered.drop(*col_drop)


In [41]:
df_l_listing_filtered.show()

+------+-------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+
|    id|host_id|         latitude|        longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|availability_365|number_of_reviews_ltm|
+------+-------+-----------------+-----------------+---------------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+
| 56766| 269557|         45.76312|          4.82813|Entire home/apt|125.0|            60|               55| 2023-10-29|             0.34|             210|                    1|
| 77104| 411285|         45.76842|          4.82745|Entire home/apt|100.0|             2|               24| 2023-10-07|             0.15|              75|                    2|
|134539| 660135|          45.7593|          4.82548|Entire home/apt| 80.0|             2|              132| 2023-12

In [42]:
df_l_listing_filtered=df_l_listing_filtered.withColumn('Hotel room',
                                     df_l_listing_filtered.room_type=='Hotel room')
df_l_listing_filtered=df_l_listing_filtered.withColumn('Entire',
                                     df_l_listing_filtered.room_type=='Entire home/apt')
df_l_listing_filtered=df_l_listing_filtered.withColumn('Private room',
                                     df_l_listing_filtered.room_type=='Private room')
df_l_listing_filtered=df_l_listing_filtered.withColumn('Shared room',
                                     df_l_listing_filtered.room_type=='Shared room')

In [43]:
import pyspark.sql.functions as F
from functools import reduce

cols = ['Hotel room', 'Entire', 'Private room', 'Shared room']
df_l_listing_filtered= reduce(lambda df_l_listing_filtered,
                              c: df_l_listing_filtered.withColumn(c, F.when(df_l_listing_filtered[c] == 'false',
                                                                            0).otherwise(1)), cols, df_l_listing_filtered)

In [44]:
df_l_listing_filtered = df_l_listing_filtered.drop('room_type')

In [45]:
df_l_listing_filtered.show()

+------+-------+-----------------+-----------------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+
|    id|host_id|         latitude|        longitude|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|availability_365|number_of_reviews_ltm|Hotel room|Entire|Private room|Shared room|
+------+-------+-----------------+-----------------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+
| 56766| 269557|         45.76312|          4.82813|125.0|            60|               55| 2023-10-29|             0.34|             210|                    1|         0|     1|           0|          0|
| 77104| 411285|         45.76842|          4.82745|100.0|             2|               24| 2023-10-07|             0.15|              75|                    2|         0|     1|      

In [56]:
df=df_l_listing_filtered

from pyspark.sql.functions import col,isnan,when,count
df = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c
                           )).alias(c)
                    for c in df.columns])
df.show()

+---+-------+--------+---------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+
| id|host_id|latitude|longitude|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|availability_365|number_of_reviews_ltm|Hotel room|Entire|Private room|Shared room|
+---+-------+--------+---------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+
|  0|      0|       0|        0|    0|             0|                0|        600|              600|               0|                    0|         0|     0|           0|          0|
+---+-------+--------+---------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+



In [60]:
from pyspark.sql.functions import col,isnan, when, count, isnull
from pyspark.sql.functions import *

In [72]:
#df2=df_l_listing_filtered
#df2 = df2.na.drop(subset=['price'])
df3 = df_l_listing_filtered.na.fill(value = 0, subset=["reviews_per_month"])
df_l_listings_cleaned = df3
df_l_listings_cleaned=df_l_listings_cleaned.drop('last_review')

In [58]:
#from pyspark.sql.functions import col,isnan,when,count
df3 = df3.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c
                           )).alias(c)
                    for c in df3.columns])
df3.show()

+---+-------+--------+---------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+
| id|host_id|latitude|longitude|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|availability_365|number_of_reviews_ltm|Hotel room|Entire|Private room|Shared room|
+---+-------+--------+---------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+
|  0|      0|       0|        0|    0|             0|                0|        600|                0|               0|                    0|         0|     0|           0|          0|
+---+-------+--------+---------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+



In [73]:
df_l_listings_cleaned.printSchema()

root
 |-- id: long (nullable = true)
 |-- host_id: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- price: double (nullable = true)
 |-- minimum_nights: long (nullable = true)
 |-- number_of_reviews: long (nullable = true)
 |-- reviews_per_month: double (nullable = false)
 |-- availability_365: long (nullable = true)
 |-- number_of_reviews_ltm: long (nullable = true)
 |-- Hotel room: integer (nullable = false)
 |-- Entire: integer (nullable = false)
 |-- Private room: integer (nullable = false)
 |-- Shared room: integer (nullable = false)



In [49]:
df_l_listing_filtered.show()

+------+-------+-----------------+-----------------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+
|    id|host_id|         latitude|        longitude|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|availability_365|number_of_reviews_ltm|Hotel room|Entire|Private room|Shared room|
+------+-------+-----------------+-----------------+-----+--------------+-----------------+-----------+-----------------+----------------+---------------------+----------+------+------------+-----------+
| 56766| 269557|         45.76312|          4.82813|125.0|            60|               55| 2023-10-29|             0.34|             210|                    1|         0|     1|           0|          0|
| 77104| 411285|         45.76842|          4.82745|100.0|             2|               24| 2023-10-07|             0.15|              75|                    2|         0|     1|      

In [74]:
#Créer un vecteur avec toutes les caractéristiques (sauf les identifiants) avec VectorAssembler
from pyspark.ml.feature import VectorAssembler

In [75]:
#Récupérer la liste des colonnes
col_list = list(df_l_listings_cleaned.columns)
col_list.remove('price')
col_list

['id',
 'host_id',
 'latitude',
 'longitude',
 'minimum_nights',
 'number_of_reviews',
 'reviews_per_month',
 'availability_365',
 'number_of_reviews_ltm',
 'Hotel room',
 'Entire',
 'Private room',
 'Shared room']

In [76]:
vectorAssembler = VectorAssembler(inputCols= col_list,
                                  outputCol = 'features')
v_df_l_listings = vectorAssembler.transform(df_l_listings_cleaned)
v_df_l_listings = v_df_l_listings.select(['features', 'price'])

In [77]:
v_df_l_listings.show()

+--------------------+-----+
|            features|price|
+--------------------+-----+
|[56766.0,269557.0...|125.0|
|[77104.0,411285.0...|100.0|
|[134539.0,660135....| 80.0|
|[174304.0,832917....|201.0|
|[233372.0,1221417...| 52.0|
|[242183.0,1271050...| 77.0|
|[249711.0,1308521...|210.0|
|[329950.0,1683652...| 75.0|
|[342674.0,1738855...|155.0|
|[349150.0,1769157...|169.0|
|[365998.0,1848025...| 76.0|
|[387453.0,718463....| 90.0|
|[551810.0,2711663...|139.0|
|[612551.0,3038226...| 70.0|
|[625569.0,3103951...|145.0|
|[626944.0,1803736...|120.0|
|[635954.0,3165326...|102.0|
|[684345.0,2719085...| 62.0|
|[695607.0,3560828...|146.0|
|[720121.0,3715660...| 54.0|
+--------------------+-----+
only showing top 20 rows



In [78]:
#Partage données d'entrainement et de test
train_df_l_listings, test_df_l_listings = v_df_l_listings.randomSplit([0.7, 0.3])

In [79]:
#Modèle de régression linéaire
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='price', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df_l_listings)
print("Coefficients:" + str(lr_model.coefficients))
print("Intercept:" + str(lr_model.intercept))

Coefficients:[5.515418921881718e-18,-5.916889530837038e-08,234.15476936561947,-211.26430222302167,-0.0664265191740292,0.0,-6.415556734488406,0.07800271709355727,-0.2441376336454392,30.89007232182576,18.183821739273597,-20.583822508610112,10.400460038180087]
Intercept:-9581.209178642031


In [81]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 83.274081
r2: 0.085474


In [82]:
train_df_l_listings.describe().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|              3329|
|   mean|116.99309101832382|
| stddev| 87.09174477262518|
|    min|              51.0|
|    max|             980.0|
+-------+------------------+



In [83]:
lr_predictions = lr_model.transform(test_df_l_listings)
lr_predictions.select("prediction","price","features").show()

+------------------+-----+--------------------+
|        prediction|price|            features|
+------------------+-----+--------------------+
|148.78965412709294| 80.0|(13,[0,1,2,3,4,7,...|
|123.90484665455733| 75.0|(13,[0,1,2,3,4,7,...|
|141.07646973677583|100.0|(13,[0,1,2,3,4,7,...|
|125.07692980599859|300.0|(13,[0,1,2,3,4,7,...|
|111.32349605662239|170.0|(13,[0,1,2,3,4,7,...|
|130.69042964554683|350.0|(13,[0,1,2,3,4,7,...|
|125.36318020925137|200.0|(13,[0,1,2,3,4,7,...|
| 128.7160478833921| 60.0|(13,[0,1,2,3,4,7,...|
|129.03078034943428|100.0|(13,[0,1,2,3,4,7,...|
|111.91747893765569|110.0|(13,[0,1,2,3,4,7,...|
|145.03483900553329| 70.0|(13,[0,1,2,3,4,7,...|
| 126.4306165774542|230.0|(13,[0,1,2,3,4,7,...|
|100.09534949603221| 75.0|(13,[0,1,2,3,4,7,...|
| 91.63084027463447|120.0|(13,[0,1,2,3,4,7,...|
| 79.84574007978881|130.0|(13,[0,1,2,3,4,7,...|
|138.24302334661843|100.0|[77104.0,411285.0...|
|153.39106249254655|210.0|[249711.0,1308521...|
|133.98282034486328|102.0|[635954.0,3165

In [84]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="price",metricName="r2")
print("R2 on test data = %g" % lr_evaluator.evaluate(lr_predictions))

R2 on test data = 0.0716597


In [85]:
test_result = lr_model.evaluate(test_df_l_listings)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 77.5783


In [50]:
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
lls = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("quote", "\"") \
  .option("escape", "\"") \
  .option("multiline", True) \
  .load(l_listings)

display(lls)

NameError: name 'file_type' is not defined

In [ ]:
#Traitement des caractères spéciaux et passage en décimal (avec 2 chiffres après la virgule)

from pyspark.sql.functions import *

#remove '$' from each string in 'price' column
df_new = df.withColumn('price', regexp_replace('price', '$', ''))
#remove all special characters
df_new = df.withColumn('price', regexp_replace('price', '[^a-zA-Z0-9]', ''))
#convert string to float
df_new = df.withColumn('price', df_new['price'].cast('float'))
#with only to digits
df_new = df.withColumn('price', round(df_new['price'], 2))
